<a href="https://colab.research.google.com/github/hjornskar-ctrl/dim-clem/blob/main/CNN_attempt_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fftshift
import tensorflow as tf

# keras imports for the dataset and building our neural network

import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

import pandas as pd
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
param_csg = np.load('/content/drive/MyDrive/csg_param_data.npy')


df = pd.DataFrame(param_csg)
df.head()



,0,1,2,3,4
0,0.282638,382.295111,56.0,0.029297,116.0
1,0.255795,380.055693,57.0,0.029297,83.0
2,0.834733,187.148172,34.0,0.048828,125.0
3,0.770519,420.454182,50.0,0.032227,159.0
4,0.909618,424.354512,52.0,0.030273,62.0


I made the parameters into a dataframe, so now I can hopefully scale the data successfully in a much simpler way. I am using z-score since it is much better for regression tasks.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

parameter_normalized_zscore = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

parameter_normalized_zscore.head()

,0,1,2,3,4
0,-0.893931,0.860082,-0.225213,-0.434271,0.509650
1,-1.004931,0.842774,-0.207989,-0.434271,-0.131175
2,1.389032,-0.648197,-0.604147,-0.046258,0.684421
3,1.123504,1.155011,-0.328559,-0.376069,1.344665
4,1.698691,1.185157,-0.294110,-0.414870,-0.538973


In [ ]:
strain_csg = np.load('/content/drive/MyDrive/csg_strain_data.npy')

First I am going to make sure there are no missing parameter values.

In [ ]:
np.isnan(param_csg).any()

False

The above shows that there are no missing parameter values.

In [ ]:


strain =np.swapaxes(strain_csg, 1, 2) #swaps the axes from (3000,3,1024) to (3000,1024,3) which is what keras wants

parameters = parameter_normalized_zscore # equating the parameter variable to the normalized score

"""

strain_train, strain_temp, parameters_train, parameters_temp = train_test_split(strain, parameters, test_size=0.2, random_state=42)

above splits the strain and parameter values into a train set and another set which will be further split into
two sets, one for validation and one for testing.

strain_val, strain_test, parameters_val, parameters_test = train_test_split(strain_temp, parameters_temp, test_size=0.5, random_state=42)

print(f"Training set:{strain_train.shape},{parameters_train.shape}")
print(f"Validation set:{strain_val.shape},{parameters_val.shape}")
print(f"Testing set:{strain_test.shape},{parameters_test.shape}")

"""

'\n\nstrain_train, strain_temp, parameters_train, parameters_temp = train_test_split(strain, parameters, test_size=0.2, random_state=42)\n\nabove splits the strain and parameter values into a train set and another set which will be further split into\ntwo sets, one for validation and one for testing.\n\nstrain_val, strain_test, parameters_val, parameters_test = train_test_split(strain_temp, parameters_temp, test_size=0.5, random_state=42)\n\nprint(f"Training set:{strain_train.shape},{parameters_train.shape}")\nprint(f"Validation set:{strain_val.shape},{parameters_val.shape}")\nprint(f"Testing set:{strain_test.shape},{parameters_test.shape}")\n\n'

In [ ]:
import numpy as np

# Assuming X.shape = (2400, 1024, 3) and y.shape = (2400, 5)

# Define split sizes
train_size = int(0.7 * len(strain))  # 70% for training
val_size = int(0.15 * len(strain))   # 15% for validation
test_size = len(strain) - train_size - val_size  # Remaining 15% for test

# Sequential split
X_train, y_train = strain[:train_size], parameters[:train_size]
X_val, y_val = strain[train_size:train_size + val_size], parameters[train_size:train_size + val_size]
X_test, y_test = strain[train_size + val_size:], parameters[train_size + val_size:]

# Print shapes to verify
print(f"Train set: X={X_train.shape}, y={y_train.shape}")
print(f"Validation set: X={X_val.shape}, y={y_val.shape}")
print(f"Test set: X={X_test.shape}, y={y_test.shape}")


Train set: X=(2100, 1024, 3), y=(2100, 5)
Validation set: X=(450, 1024, 3), y=(450, 5)
Test set: X=(450, 1024, 3), y=(450, 5)


I have split the data into three different sets:

The training set, validation set, and testing set.

The model will learn patterns from the labelled data in the training set, improving the weights and biases in the CNN using a loss function like MSE. The hyperparameters of the model will be tuned using the test set.

Then the model will be tested on the test set (which is unseen data), where the obtained values can be compared to the real ones.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, BatchNormalization, ReLU, Add, GlobalAveragePooling1D, Dense, Input, MaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout

def residual_block(x, filters, kernel_size=3, stride=1):
    """Residual block for 1D ResNet."""
    shortcut = x  # Skip connection

    # First Conv1D layer
    x = Conv1D(filters, kernel_size, strides=stride, padding="same",kernel_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Second Conv1D layer
    x = Conv1D(filters, kernel_size, strides=1, padding="same",kernel_regularizer=l2(0.0001))(x)  # Stride=1 to maintain size
    x = BatchNormalization()(x)

    # Skip Connection: Match dimensions if needed
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv1D(filters, 1, strides=stride, padding="same",kernel_regularizer=l2(0.0001))(shortcut)  # 1x1 conv to match dimensions
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])  # Add skip connection
    x = ReLU()(x)  # Final activation

    return x

def build_resnet_1d(input_shape, num_outputs):
    """Builds a ResNet-based 1D CNN model for time-series parameter estimation."""
    inputs = Input(shape=input_shape)  # (1024, 3)

    # Initial Conv Layer
    x = Conv1D(64, kernel_size=7, strides=2, padding="same",kernel_regularizer=l2(0.0001))(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling1D(pool_size=3, strides=2, padding="same")(x)  # Reduce temporal size

    # Residual Blocks
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, stride=2)  # Downsample
    x = residual_block(x, filters=256, stride=2)  # Downsample
    x = residual_block(x, filters=512, stride=2)  # Downsample

    # Global Average Pooling & Fully Connected Layers
    x=Dropout(0.2)(x)
    x = GlobalAveragePooling1D()(x)  # Reduces (time, filters) to (filters,)
    x = Dropout(0.3)(x)
    outputs = Dense(num_outputs, activation="linear",kernel_regularizer=l2(0.0001))(x)  # Predicts 5 parameters (regression)

    model = Model(inputs, outputs)
    return model

# Model initialization
input_shape = (1024, 3)  # 1024 time steps, 3 features
num_outputs = 5  # Predict 5 continuous parameters
resnet_1d = build_resnet_1d(input_shape, num_outputs)
resnet_1d.summary()


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9             │ (None, 1024, 3)        │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_108 (Conv1D)       │ (None, 512, 64)        │          1,408 │ input_layer_9[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_108   │ (None, 512, 64)        │            256 │ conv1d_108[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_81 (ReLU)           │ (None, 512, 64)        │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_9           │ (None, 256, 64)        │              0 │ re_lu_81[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_109 (Conv1D)       │ (None, 256, 64)        │         12,352 │ max_pooling1d_9[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_109   │ (None, 256, 64)        │            256 │ conv1d_109[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_82 (ReLU)           │ (None, 256, 64)        │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_110 (Conv1D)       │ (None, 256, 64)        │         12,352 │ re_lu_82[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_110   │ (None, 256, 64)        │            256 │ conv1d_110[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_36 (Add)              │ (None, 256, 64)        │              0 │ batch_normalization_1… │
│                           │                        │                │ max_pooling1d_9[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_83 (ReLU)           │ (None, 256, 64)        │              0 │ add_36[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_111 (Conv1D)       │ (None, 128, 128)       │         24,704 │ re_lu_83[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_111   │ (None, 128, 128)       │            512 │ conv1d_111[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_84 (ReLU)           │ (None, 128, 128)       │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_112 (Conv1D)       │ (None, 128, 128)       │         49,280 │ re_lu_84[0][0]         │
├──────────────────────

 Total params: 1,763,205 (6.73 MB)

 Trainable params: 1,757,445 (6.70 MB)

 Non-trainable params: 5,760 (22.50 KB)

Above is the resnet architecture which has been modified to have an input size of (1024,3), and an output of 5 parameters. Will make a diagram of the architecture later.

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


# Compile the model
optimizer = Adam(learning_rate=1e-5) # reduced learning rate to allow for convergence
resnet_1d.compile(optimizer=optimizer, loss="mse", metrics=["mae"])

# Early stopping (stops training if val_loss doesn't improve for 7 epochs)
early_stop = EarlyStopping(monitor="val_loss", patience=25, restore_best_weights=True, verbose=1)


# Train on (2400, 1024, 3) inputs and (2400, 5) labels
resnet_1d.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=258,callbacks=[early_stop])

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - loss: 0.1424 - mae: 0.0984 - val_loss: 0.3634 - val_mae: 0.3132
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - loss: 0.1406 - mae: 0.0952 - val_loss: 0.3641 - val_mae: 0.3128
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.1408 - mae: 0.0980 - val_loss: 0.3641 - val_mae: 0.3142
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.1413 - mae: 0.0952 - val_loss: 0.3632 - val_mae: 0.3136
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.1386 - mae: 0.0948 - val_loss: 0.3636 - val_mae: 0.3139
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.1387 - mae: 0.0972 - val_loss: 0.3614 - val_mae: 0.3123
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.1401 - mae: 0.0967 - val_loss: 0.3620 - val_mae: 0.3136
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.1382 - mae: 0.0925 - val_loss: 0.3622 - val_mae: 0.3136
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.1396 - mae: 0.0965 - v

The loss on the validation is much higher than the loss on the training data, which indicates that it is overfitting to the training data. To alleviate this, regularisation must be used.

## 1st Run

The final mse value for the training loss (MSE) was 0.0325 and the MAE was 0.1209, while for the validation it was 0.1911 and 0.2918 respectively. On my 2nd run I am reducing the learning rate of the Adam optimiser to 1e-4.

##2nd Run

On the second run, the final training loss values were 0.0375 and 0.1315, while the final validation loss and MAE was 0.2327 and 0.3179. Adam learning rate was 1e-4.

##3rd Run

On the third run with 1e-5, the values were:

loss: 0.0196

mae: 0.0878

val_loss: 0.2134

val_mae: 0.3040.

Definitely overfitting now. Will add regularisation.

## 4th Run

In addition to reducing the learning rate, an L2 kernal regulariser was introduced.

This made it underfit the training data, indicating the regularisation is too strong:

loss: 2.1170

mae: 0.2230 -

val_loss: 2.2906

val_mae: 0.3591

Will reduce the regularisation hyperparameter.

## 5th Run

loss: 0.3385 - mae: 0.2196 - val_loss: 0.4895 - val_mae: 0.3451

reduced it from 0.01 to 0.001 and it is still overfitting, so i'm going to add a dropout of 0.3.

## 6th Run

loss: 0.2765 - mae: 0.2048 - val_loss: 0.4574 - val_mae: 0.3535

Still overfitting so I'm going to use learning rate decay.

## 7th RUn

loss: 0.2034 - mae: 0.1699 - val_loss: 0.3908 - val_mae: 0.3142

A bit better after adding the scheduler, however not amazing. Going to increase the patience so it starts the scheduler later. Going to increase it from 3 to 5.

## 8th Run

0.1546 - mae: 0.1454 - val_loss: 0.3627 - val_mae: 0.3125

Going to add early stopping.

0.1518 - mae: 0.1147 - val_loss: 0.3606 - val_mae: 0.3110



0.1440 - mae: 0.1039 - val_loss: 0.3611 - val_mae: 0.3139

The smallest difference I could get was ~0.2. I think i must add data augmentation :(